# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2022 9:58AM,237839,10,0085695182,ISDIN Corporation,Released
1,May 23 2022 9:58AM,237839,10,0085696323,ISDIN Corporation,Released
2,May 23 2022 9:58AM,237839,10,0085696430,ISDIN Corporation,Released
3,May 23 2022 9:56AM,237837,16,EMERSA-8051750,"Emersa Waterbox, LLC",Released
4,May 23 2022 9:53AM,237830,10,0085696431,ISDIN Corporation,Released
5,May 23 2022 9:53AM,237830,10,0085696435,ISDIN Corporation,Released
6,May 23 2022 9:53AM,237830,10,0085696437,ISDIN Corporation,Released
7,May 23 2022 9:53AM,237830,10,0085696441,ISDIN Corporation,Released
8,May 23 2022 9:53AM,237830,10,0085696439,ISDIN Corporation,Released
9,May 23 2022 9:53AM,237830,10,0085696444,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,237829,Released,1
38,237830,Released,63
39,237832,Released,1
40,237837,Released,1
41,237839,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237829,NaN,NaN,1.0
237830,NaN,NaN,63.0
237832,NaN,NaN,1.0
237837,NaN,NaN,1.0
237839,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237774,1.0,30.0,3.0
237781,0.0,0.0,51.0
237787,0.0,0.0,1.0
237789,1.0,6.0,0.0
237792,0.0,0.0,18.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237774,1,30,3
237781,0,0,51
237787,0,0,1
237789,1,6,0
237792,0,0,18


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237774,1,30,3
1,237781,0,0,51
2,237787,0,0,1
3,237789,1,6,0
4,237792,0,0,18


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237774,1,30,3
1,237781,,,51
2,237787,,,1
3,237789,1,6,
4,237792,,,18


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2022 9:58AM,237839,10,ISDIN Corporation
3,May 23 2022 9:56AM,237837,16,"Emersa Waterbox, LLC"
4,May 23 2022 9:53AM,237830,10,ISDIN Corporation
67,May 23 2022 9:48AM,237832,19,"Bi-Coastal Pharma International, LLC"
68,May 23 2022 9:46AM,237829,18,"ClearSpec, LLC"
69,May 23 2022 9:46AM,237826,10,ISDIN Corporation
110,May 23 2022 9:44AM,237828,10,"CLINUVEL, Inc."
111,May 23 2022 9:41AM,237825,18,"ClearSpec, LLC"
112,May 23 2022 9:40AM,237824,10,"Methapharm, Inc."
117,May 23 2022 9:40AM,237822,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2022 9:58AM,237839,10,ISDIN Corporation,,,3
1,May 23 2022 9:56AM,237837,16,"Emersa Waterbox, LLC",,,1
2,May 23 2022 9:53AM,237830,10,ISDIN Corporation,,,63
3,May 23 2022 9:48AM,237832,19,"Bi-Coastal Pharma International, LLC",,,1
4,May 23 2022 9:46AM,237829,18,"ClearSpec, LLC",,,1
5,May 23 2022 9:46AM,237826,10,ISDIN Corporation,,,41
6,May 23 2022 9:44AM,237828,10,"CLINUVEL, Inc.",,,1
7,May 23 2022 9:41AM,237825,18,"ClearSpec, LLC",,,1
8,May 23 2022 9:40AM,237824,10,"Methapharm, Inc.",,,5
9,May 23 2022 9:40AM,237822,10,"Methapharm, Inc.",,,21


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 9:58AM,237839,10,ISDIN Corporation,3,,
1,May 23 2022 9:56AM,237837,16,"Emersa Waterbox, LLC",1,,
2,May 23 2022 9:53AM,237830,10,ISDIN Corporation,63,,
3,May 23 2022 9:48AM,237832,19,"Bi-Coastal Pharma International, LLC",1,,
4,May 23 2022 9:46AM,237829,18,"ClearSpec, LLC",1,,
5,May 23 2022 9:46AM,237826,10,ISDIN Corporation,41,,
6,May 23 2022 9:44AM,237828,10,"CLINUVEL, Inc.",1,,
7,May 23 2022 9:41AM,237825,18,"ClearSpec, LLC",1,,
8,May 23 2022 9:40AM,237824,10,"Methapharm, Inc.",5,,
9,May 23 2022 9:40AM,237822,10,"Methapharm, Inc.",21,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 9:58AM,237839,10,ISDIN Corporation,3,,
1,May 23 2022 9:56AM,237837,16,"Emersa Waterbox, LLC",1,,
2,May 23 2022 9:53AM,237830,10,ISDIN Corporation,63,,
3,May 23 2022 9:48AM,237832,19,"Bi-Coastal Pharma International, LLC",1,,
4,May 23 2022 9:46AM,237829,18,"ClearSpec, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 9:58AM,237839,10,ISDIN Corporation,3.0,NaN,NaN
1,May 23 2022 9:56AM,237837,16,"Emersa Waterbox, LLC",1.0,NaN,NaN
2,May 23 2022 9:53AM,237830,10,ISDIN Corporation,63.0,NaN,NaN
3,May 23 2022 9:48AM,237832,19,"Bi-Coastal Pharma International, LLC",1.0,NaN,NaN
4,May 23 2022 9:46AM,237829,18,"ClearSpec, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 9:58AM,237839,10,ISDIN Corporation,3.0,0.0,0.0
1,May 23 2022 9:56AM,237837,16,"Emersa Waterbox, LLC",1.0,0.0,0.0
2,May 23 2022 9:53AM,237830,10,ISDIN Corporation,63.0,0.0,0.0
3,May 23 2022 9:48AM,237832,19,"Bi-Coastal Pharma International, LLC",1.0,0.0,0.0
4,May 23 2022 9:46AM,237829,18,"ClearSpec, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4280656,371.0,35.0,1.0
15,1426797,80.0,0.0,0.0
16,237837,1.0,0.0,0.0
18,2140265,44.0,8.0,0.0
19,475621,1.0,6.0,1.0
22,237787,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4280656,371.0,35.0,1.0
1,15,1426797,80.0,0.0,0.0
2,16,237837,1.0,0.0,0.0
3,18,2140265,44.0,8.0,0.0
4,19,475621,1.0,6.0,1.0
5,22,237787,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,371.0,35.0,1.0
1,15,80.0,0.0,0.0
2,16,1.0,0.0,0.0
3,18,44.0,8.0,0.0
4,19,1.0,6.0,1.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,371.0
1,15,Released,80.0
2,16,Released,1.0
3,18,Released,44.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,22
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,1.0,0.0
Executing,35.0,0.0,0.0,8.0,6.0,0.0
Released,371.0,80.0,1.0,44.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,22
0,Completed,1.0,0.0,0.0,0.0,1.0,0.0
1,Executing,35.0,0.0,0.0,8.0,6.0,0.0
2,Released,371.0,80.0,1.0,44.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,22
0,Completed,1.0,0.0,0.0,0.0,1.0,0.0
1,Executing,35.0,0.0,0.0,8.0,6.0,0.0
2,Released,371.0,80.0,1.0,44.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()